In [ ]:
import sys
sys.path.append('/home/jh20/narin/physionet/python-classifier-2022')

In [ ]:
from helper_code import *

In [ ]:
import argparse, warnings, sys
import torch

sys.path.append('/home/jh20/narin/dcase/DESED_task')
sys.path.append('/home/jh20/narin/dcase/DESED_task/recipes/dcase2022_task4_baseline')


parser = argparse.ArgumentParser(description = "HMD_trainer")
## Training Settings
parser.add_argument('--train_data_folder',    default='/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train/',     help='train data path')
parser.add_argument('--valid_data_folder',    default='/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/validation/',     help='validation data path')
parser.add_argument('--output_folder',    default='./output_folder',     help='output folder')
parser.add_argument("--label_type",default="murmur", type=str, help="label type")
parser.add_argument("--lr",default=1e-3, type=float, help="learning rate")
parser.add_argument("--minibatchsize_train", default=64, type=int)
parser.add_argument("--minibatchsize_valid", default=1, type=int)
parser.add_argument("--train_num_workers", default=6, type=int, help="number of training workers")
parser.add_argument("--dev_num_workers", default=1, type=int, help="number of validation workers")
parser.add_argument("--seed", default=617, type=int)
parser.add_argument("--model_path", default='./hmd_CNN', type=str)
parser.add_argument("--logdir", default='./log_hmd/', type=str)
parser.add_argument("--model_name", default='hmd_CNN_model', type=str)
parser.add_argument("--start_iter", default=0, type=int)
parser.add_argument("--end_iter", default=200, type=int)
parser.add_argument("--gpu", default="0", type=str)




warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
warnings.simplefilter("ignore")
torch.multiprocessing.set_sharing_strategy('file_system')
args = parser.parse_args(args=[])

In [ ]:
def load_challenge_model(model_folder, verbose):
    murmur_train_model = torch.load('/home/jh20/narin/physionet/hmd_CNN' + '/' + 'hmd_CNN_model_40.model')
    outcome_train_model = torch.load('/home/jh20/narin/physionet/hmd_CNN_outcome' + '/' + 'hmd_CNN_outcome_model_25.model')
    return [murmur_train_model, outcome_train_model]

In [ ]:
import librosa
import math
import random

def get_features(data, recordings):
    
    murmur_classes = ['Present', 'Unknown', 'Absent']
    num_murmur_classes = len(murmur_classes)
    outcome_classes = ['Abnormal', 'Normal']
    num_outcome_classes = len(outcome_classes)
    
    age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']
    
    num_recordings = len(recordings)
    
    
    feature_dict={}
    
    feature_dict['mel']=[]
    for i in range(num_recordings):
        #log mel feature
        log_mel_feature = librosa.power_to_db(librosa.feature.melspectrogram(y = recordings[i].astype(np.float32),
                                                         sr= 4000,
                                                             n_mels=128,
                                                             n_fft=400, 
                                                             hop_length=128, 
                                                             win_length=400))
        feature_dict['mel'].append(log_mel_feature)

    # age
    current_patient_age = get_age(data)
    current_age_group = np.zeros(6, dtype=np.float32)
    if current_patient_age in age_classes:
        j = age_classes.index(current_patient_age)
        current_age_group[j] = 1.0
    else :
        current_age_group[5] = 1.0

    feature_dict['age']=current_age_group



    # sex
    sex = get_sex(data)
    sex_feature = np.zeros(2, dtype=np.float32)
    if compare_strings(sex, 'Female'):
        sex_feature[0] = 1.0
    elif compare_strings(sex, 'Male'):
        sex_feature[1] = 1.0

    feature_dict['sex']=sex_feature

    # height and weight.
    height = get_height(data)
    weight = get_weight(data)

    ## simple impute
    if math.isnan(height) :
        height = 110.846  #mean
    if math.isnan(weight) :
        weight = 23.767   #mean

    height_weight = np.array([height, weight], dtype=np.float32)


    feature_dict['hw']=height_weight

    # Extract pregnancy
    preg_feature = np.zeros(2, dtype=np.float32)
    is_pregnant = get_pregnancy_status(data)
    if is_pregnant == True:
        preg_feature[0] = 1.0
    elif is_pregnant == False:
        preg_feature[1] = 1.0

    feature_dict['preg']=preg_feature

    # Extract location
    feature_dict['loc']=[]
    locations = get_locations(data)
    for j in range(num_recordings):
        
        num_recording_locations = len(recording_locations)
        loc_feature = np.zeros(num_recording_locations, dtype=np.float32)
        if locations[j] in recording_locations:
            idx = recording_locations.index(locations[j])
            loc_feature[idx] = 1.0
            
        feature_dict['loc'].append(loc_feature)




    # label

    current_murmur = np.zeros(num_murmur_classes, dtype=np.float32)
    murmur = get_murmur(data)
    if murmur in murmur_classes:
        j = murmur_classes.index(murmur)
        current_murmur[j] = 1
    
    feature_dict['murmur']=current_murmur
    
    current_outcome = np.zeros(num_outcome_classes, dtype=np.float32)
    outcome = get_outcome(data)
    if outcome in outcome_classes:
        j = outcome_classes.index(outcome)
        current_outcome[j] = 1

    feature_dict['outcome']=current_outcome
    
    


    return feature_dict

In [ ]:
import sys
sys.path.append("/home/jh20/narin/physionet/python-classifier-2022/model")

from CNN_outcome import CNN as CNN_outcome
from CNN_murmur import CNN as CNN_murmur


def run_challenge_model(model, data, recordings, verbose):
    murmur_model , outcome_model = model
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     print ('Available devices ', torch.cuda.device_count())
        
    GPU_NUM = args.gpu # 원하는 GPU 번호 입력
    device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
    torch.cuda.set_device(device) # change allocation of current GPU

    
    # Define the model
    murmur_classifier = CNN_murmur()
    murmur_classifier = murmur_classifier.cuda(device)
    
    outcome_classifier = CNN_outcome()
    outcome_classifier = outcome_classifier.cuda(device)
    
    murmur_classifier.load_state_dict(murmur_model)
    outcome_classifier.load_state_dict(outcome_model)
    
#     wav_files = glob.glob(data_folder + '*.wav')
#     num_wav_files = len(wav_files)
    
    murmur_classes = ['Present', 'Unknown', 'Absent']
    num_murmur_classes = len(murmur_classes)
    outcome_classes = ['Abnormal', 'Normal']
    num_outcome_classes = len(outcome_classes)
    
    age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']
    
    num_recordings = len(recordings)
    recordings_list=[]
    for i in range(num_recordings):
        recording = recordings[i]
        length = 20*4000  
        if recording.shape[0] <= length:
            shortage = length - recording.shape[0]
            recording = np.pad(recording, (0, shortage), 'wrap')
        start_frame = np.int64(random.random()*(recording.shape[0]-length))
        recording = recording[start_frame:start_frame + length] 
        
        recordings_list.append(recording)
        

    features = get_features(data, recordings_list)
    
    predict_murmur_arr= np.zeros((num_recordings,3))
    predict_outcome_arr= np.zeros((num_recordings,2))
    
    for i in range(num_recordings):
        
        logmel = torch.tensor(features['mel'][i]).cuda()
        age = torch.tensor(features['age']).cuda()
        sex = torch.tensor(features['sex']).cuda()
        hw = torch.tensor(features['hw']).cuda()
        preg = torch.tensor(features['preg']).cuda()
        loc = torch.tensor(features['loc'][i]).cuda()
        
        logmel = logmel.unsqueeze(0)
        age = age.unsqueeze(0)
        sex = sex.unsqueeze(0)
        hw = hw.unsqueeze(0)
        preg = preg.unsqueeze(0)
        loc = loc.unsqueeze(0)
        
        
        predict_murmur = murmur_classifier(logmel,age,sex,hw,preg,loc)
        predict_outcome = outcome_classifier(logmel,age,sex,hw,preg,loc)
        
        predict_murmur_arr[i,:]= predict_murmur.data.detach().cpu().numpy()
        predict_outcome_arr[i,:] = predict_outcome.data.detach().cpu().numpy()

    
    
    
   
    # Get classifier probabilities.
    idx1 = predict_murmur_arr.argmax(axis=0)[0]
    murmur_probabilities = predict_murmur_arr[idx1,] 
    idx2 = predict_outcome_arr.argmax(axis=0)[0]
    outcome_probabilities = predict_outcome_arr[idx2,]


    # Choose label with highest probability.
    murmur_labels = np.zeros(len(murmur_classes), dtype=np.int_)
    idx = np.argmax(murmur_probabilities)
    murmur_labels[idx] = 1
    outcome_labels = np.zeros(len(outcome_classes), dtype=np.int_)
    idx = np.argmax(outcome_probabilities)
    outcome_labels[idx] = 1

    # Concatenate classes, labels, and probabilities.
    classes = murmur_classes + outcome_classes
    labels = np.concatenate((murmur_labels, outcome_labels))
    probabilities = np.concatenate((murmur_probabilities, outcome_probabilities))

    return classes, labels, probabilities

In [ ]:
import numpy as np, os, sys
# from helper_code import *
# from team_code import load_challenge_model, run_challenge_model

# Run model.
def run_model(model_folder, data_folder, output_folder, allow_failures, verbose):
    # Load models.
    if verbose >= 1:
        print('Loading Challenge model...')

    model = load_challenge_model(model_folder, verbose) ### Teams: Implement this function!!!

    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the Challenge outputs if it does not already exist.
    os.makedirs(output_folder, exist_ok=True)

    # Run the team's model on the Challenge data.
    if verbose >= 1:
        print('Running model on Challenge data...')

    # Iterate over the patient files.
    for i in range(num_patient_files):
        if verbose >= 2:
            print('    {}/{}...'.format(i+1, num_patient_files))

        patient_data = load_patient_data(patient_files[i])
        recordings = load_recordings(data_folder, patient_data)

        # Allow or disallow the model to fail on parts of the data; helpful for debugging.
        try:
            classes, labels, probabilities = run_challenge_model(model, patient_data, recordings, verbose) ### Teams: Implement this function!!!
        except:
            if allow_failures:
                if verbose >= 2:
                    print('... failed.')
                classes, labels, probabilities = list(), list(), list()
            else:
                raise

        # Save Challenge outputs.
        head, tail = os.path.split(patient_files[i])
        root, extension = os.path.splitext(tail)
        output_file = os.path.join(output_folder, root + '.csv')
        patient_id = get_patient_id(patient_data)
        save_challenge_outputs(output_file, patient_id, classes, labels, probabilities)

    if verbose >= 1:
        print('Done.')